In [ ]:







import os
import zipfile
import imageio
from skimage.filters import gaussian
from scipy.ndimage import median_filter
from skimage.morphology import disk
from skimage.restoration import denoise_nl_means
from skimage.transform import resize


zip_path = 'Train Image Dataseta11bf6d.zip'

# Specify the path to the output directory
output_dir = 'preprocessed_images'

# Create the output directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Open the zip file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    # Get a list of file names in the zip file
    image_files = [f for f in zip_ref.namelist() if f.endswith('.jpg') or f.endswith('.png')]

    # Iterate over the image files and preprocess each image
    for file_name in image_files:
        # Read the image from the zip file
        with zip_ref.open(file_name) as image_file:
            image = imageio.imread(image_file)

        # Resize the image to a smaller size for faster processing
        resized_image = resize(image, (512, 512), preserve_range=True)

        # Apply preprocessing techniques
        # Gaussian blur
        blurred_image = gaussian(resized_image, sigma=2, preserve_range=True)

        # Median filtering using scipy.ndimage.median_filter
        median_filtered_image = median_filter(resized_image, size=3, mode='reflect')

        # Denoising using non-local means
        denoised_image = denoise_nl_means(resized_image, patch_size=5, patch_distance=3, h=0.1, fast_mode=False, preserve_range=True)

        # Extract the filename and subdirectories from the file path
        directory, filename = os.path.split(file_name)
        subdirectories = directory.split('/')

        # Create the subdirectories within the output directory to save the preprocessed images
        output_subdir = os.path.join(output_dir, *subdirectories)
        if not os.path.exists(output_subdir):
            os.makedirs(output_subdir)

        # Save the preprocessed images to the output subdirectory
        imageio.imsave(os.path.join(output_subdir, f'blurred_{filename}'), blurred_image.astype(image.dtype))
        imageio.imsave(os.path.join(output_subdir, f'median_filtered_{filename}'), median_filtered_image.astype(image.dtype))
        imageio.imsave(os.path.join(output_subdir, f'denoised_{filename}'), denoised_image.astype(image.dtype))

        
        

C:\Users\SK VERMA\AppData\Local\Temp\ipykernel_46044\1656813651.py:29: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(image_file)
C:\Users\SK VERMA\AppData\Local\Temp\ipykernel_46044\1656813651.py:36: FutureWarning: Automatic detection of the color channel was deprecated in v0.19, and `channel_axis=None` will be the new default in v0.21. Set `channel_axis=-1` explicitly to silence this warning.
  blurred_image = gaussian(resized_image, sigma=2, preserve_range=True)


In [ ]:
import os
import cv2
import numpy as np

# Specify the path to the preprocessed images directory
preprocessed_dir = 'preprocessed_images'

# Specify the path to the output directory for extracted features
output_dir = 'extracted_features'

# Create the output directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Iterate over the preprocessed image files
for root, dirs, files in os.walk(preprocessed_dir):
    for file_name in files:
        # Read the preprocessed image
        image_path = os.path.join(root, file_name)
        image = cv2.imread(image_path)

        # Perform feature extraction (e.g., SIFT, SURF, or CNN-based feature extraction)
        # Example using SIFT
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        sift = cv2.SIFT_create()
        keypoints, descriptors = sift.detectAndCompute(gray, None)

        # Save the extracted features
        feature_file = os.path.join(output_dir, f'{file_name}.npy')
        np.save(feature_file, descriptors)

        print(f"Features extracted for {file_name}")

print("Feature extraction completed.")
